# Visualizing Results

## Author: Luis Eduardo Ferro Diez, <a href="mailto:luisedof10@gmail.com">luisedof10@gmail.com</a>

This notebook contains some presentation logic to visualize the results after they are processed with the projec's tool.

### Visualization Function
This function will display a leaflet map with the computed clusters and the corresponding categories.

In [1]:
import json
import folium
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import random
from shapely import wkt
from shapely.geometry import mapping
from itertools import cycle


def plot_clusters_on_map(clusters,
                              remove_noise=True,
                              print_classes=False,
                              center=(34, -118), 
                              zoom=10, 
                              tiles="OpenStreetMap",
                              score_threshold=0.1,
                              display_clusters=[]):
    unique_clusters = np.unique(clusters.cluster.values)
    n_clusters = unique_clusters.shape[0]
    colormap = cycle(plt.cm.rainbow(np.linspace(0, 1, n_clusters)))
    colors = [matplotlib.colors.to_hex(next(colormap)) for _ in range(len(unique_clusters))]
    random.shuffle(colors)
    colors = {k: c for k, c in zip(unique_clusters, colors)}
    
    m = folium.Map(location=center, 
                zoom_start=zoom,
                tiles=tiles)
    
    def style_function(row):
        color = colors[row.cluster]
        return lambda x :{'fillColor': color,
                         'fillOpacity': 0.6,
                         'weight': 1}
    
    def highlight_function(row):
        color = colors[row.cluster]
        return lambda x :{'fillColor': color,
                         'fillOpacity': 0.8,
                         'weight': 1}
    
    def create_tooltip(row):
        cluster_id = row['cluster']
        size = row['size']
        permanent = True if cluster_id in display_clusters else False
        if print_classes:
            classes = row.predictions
            classes = [ f"{cl} - {score*100:.2f}%" for cl, score in classes if score >= score_threshold]
            classes = '<br>'.join(classes)
            return folium.map.Tooltip(text=f"Cluster id: {cluster_id}<br>Size: {size}<br>{classes}",
                                        style="color: DodgerBlue", permanent=permanent)
        else:
            return folium.map.Tooltip(text=f"Cluster id: {cluster_id}<br>Size: {size}",
                                    style="color: DodgerBlue", permanent=permanent)
    def create_polygon(row):
        geojson = json.dumps(mapping(wkt.loads(row['polygon'])))
        color = colors[row.cluster]
        tooltip = create_tooltip(row)
        
        folium.GeoJson(geojson,
                       style_function=style_function(row),
                       highlight_function=highlight_function(row),
                       tooltip=tooltip).add_to(m)
    
    clusters.apply(create_polygon, axis=1)
    
    return m

Load one some sample results for visualization.
### Classified clusters in Los Angeles

In [15]:
import pandas as pd

clusters = pd.read_csv("../../../samples/LA_classified_clusters.csv")
clusters['predictions'] = clusters['predictions'].apply(eval)
clusters.head()

,cluster,size,polygon,corpus,d2v_embeddings,predictions
0,0,16,"POLYGON ((-118.30152875 33.72037932, -118.3005...",@jojosantiago29 well it's only awkward if you ...,[-0.09688469 -0.24729645 -0.698641 0.228461...,"[(Clothing, Shoes & Jewelry, 0.565152227878570..."
1,1,14,"POLYGON ((-118.28882255 33.7183059, -118.28529...",3 MINUTES #TRYHARD\na boy named nolan took his...,[-0.06338427 -0.06622423 -0.42020383 -0.150844...,"[(Movies & TV, 0.3905380666255951), (Music, 0...."
2,2,4,"POLYGON ((-118.30774141 33.75069469, -118.3127...",Frank Sinatra is a very wonderful man.\nIf she...,[ 1.24486573e-01 1.95163086e-01 -3.64264250e-...,"[(Books, 0.4513542354106903), (Music, 0.251094..."
3,3,13,"POLYGON ((-118.30599123 33.78586724, -118.3020...",@_aarriii thank you 😁\n@david_e_the_g I hate w...,[ 9.95174646e-02 8.67639948e-03 -3.89716059e-...,"[(Books, 0.6354166865348816), (Music, 0.122283..."
4,4,14,"POLYGON ((-118.29946722 33.79647899, -118.3009...",“@Liddo_Loops: some people just act all bad as...,[ 4.03775135e-03 4.63746488e-02 -4.33946103e-...,"[(Movies & TV, 0.21180206537246704), (Music, 0..."


In [16]:
# These cluster ids will have the tooltip permanent
display_clusters = [29, 50, 86, 53, 146, 156]
plot_clusters_on_map(clusters, print_classes=True, display_clusters=display_clusters)

### Classified clusters in New York

In [6]:
clusters = pd.read_csv("../../../samples/NY_classified_clusters.csv")
clusters['predictions'] = clusters['predictions'].apply(eval)
clusters.head()

,cluster,size,polygon,corpus,d2v_embeddings,predictions
0,0,12,"POLYGON ((-78.66741187 42.92364949, -78.667453...",Don't talk to me..\nGo ahead. I don't care.\nC...,[ 6.44158423e-02 -1.22264467e-01 -1.68181643e-...,"[(Music, 0.9612022638320923), (Movies & TV, 0...."
1,1,10,"POLYGON ((-78.87448771 43.01256207, -78.874005...",I need some plans that don't consist of me hav...,[ 1.87617809e-01 1.26122404e-02 -1.03301607e-...,"[(Books, 0.2173667997121811), (Health & Person..."
2,2,10,"POLYGON ((-78.88289935 43.02130306, -78.883073...","“@matchMy_SWAG: Ballons are so weird ""Happy Bi...",[ 0.03296842 0.01928129 -0.2746797 0.116389...,"[(Toys & Games, 0.31276047229766846), (Books, ..."
3,3,7,"POLYGON ((-78.8944673 43.006284, -78.89121088 ...","Tell me, do you really wanna love me forever?\...",[ 0.2753385 0.04572586 -0.1882219 -0.185345...,"[(Toys & Games, 0.4259955883026123), (Music, 0..."
4,4,12,"POLYGON ((-78.89742758 43.00905252, -78.900521...",@80Wesley @jeffnoworyta because this world as ...,[ 0.26212916 -0.01169884 -0.38635436 0.072064...,"[(Toys & Games, 0.35767650604248047), (Sports ..."


In [9]:
display_clusters = [52, 59, 74, 36, 24, 41]
plot_clusters_on_map(clusters, print_classes=True, center=(40.7, -74.0), display_clusters=display_clusters)

### Classified clusters in Vancouver

In [10]:
clusters = pd.read_csv("../../../samples/VA_classified_clusters.csv")
clusters['predictions'] = clusters['predictions'].apply(eval)
clusters.head()

,cluster,size,polygon,corpus,d2v_embeddings,predictions
0,0,17,"POLYGON ((-123.14643655 49.1707784, -123.14208...","Kurangi ngadu manuk, perbanyak ibadah! Itu kun...",[-1.56114161e-01 -9.14138481e-02 -1.25550881e-...,"[(Home & Kitchen, 0.2800077497959137), (Techno..."
1,1,13,"POLYGON ((-123.08296675 49.26252349, -123.0871...",going to work now...\n3 BF Interval Circuits o...,[ 1.78477049e-01 -9.07777324e-02 -2.30810538e-...,"[(Sports & Outdoors, 0.5594919919967651), (Clo..."
2,2,421,"POLYGON ((-123.09125141 49.26621433, -123.0953...",@tooti1919 لبي قلبك يالغالي 😉\nUk system more ...,[ 4.32584673e-01 7.77660683e-03 7.86323100e-...,"[(Home & Kitchen, 0.4004102051258087), (Sports..."
3,3,13,"POLYGON ((-123.101 49.2451, -123.101184 49.250...",@anem018ctw ラノベ結構好きだったり…！\n\nちょこちょこしか読んでないけどw...,[ 1.30206913e-01 3.30309793e-02 -2.52786309e-...,"[(Sports & Outdoors, 0.5331861972808838), (Clo..."
4,4,13,"POLYGON ((-123.02830186 49.24574315, -123.0326...","Не успеваю улаживать волосы термоприборами, аж...",[-5.47164045e-02 9.00261253e-02 -4.31964874e-...,"[(Music, 0.6213393211364746), (Toys & Games, 0..."


In [13]:
display_clusters = [0, 1, 2, 4, 6, 9]
plot_clusters_on_map(clusters, print_classes=True, center=(49.2, -123.1), display_clusters=display_clusters)